In [1]:
import gym
from myrl.environments import Envs
from myrl.policies import GaussianPolicy
from myrl.value_functions import ValueFunctionMLP, polyak
from myrl.visualizer import showit
from myrl.utils import ExperimentWriter, check_output, global_gradient_clip, list_gae, gae, add_list2list
from myrl.buffers import ReplayBuffer
from myrl.exploration import RandomNetworkBonus
import torch
import copy
import numpy as np

# envname = 'LunarLanderContinuous-v2'
envname = 'Pendulum-v0'

wll = ExperimentWriter('tb/ppo_pen_works')
env = gym.make(envname)
idim = env.observation_space.shape[0]
odim = env.action_space.shape[0]
envs = Envs(envname, 128)
envs.evenout(200)
idim, odim


(3, 1)

In [10]:
def objective_function(trial):
    env = gym.make(envname)
    envs = Envs(envname, 128)
    envs.evenout(200)
    seed = trial.suggest_int("seed", 0, 5)
    np.random.seed(seed)
    envs.seed(seed)
    torch.manual_seed(seed)
    eps = 50
    lr_policy = trial.suggest_loguniform("lr_policy", 1e-4, 5e-4)
    lr_vfunc  = trial.suggest_loguniform("lr_vfunc", 1e-4, 1e2)
    w1_pi     = trial.suggest_int("width_layer1_policy", 8, 128)
    #w2_pi     = trial.suggest_int("width_layer2_policy", 1, 64)
    w1_vfunc  = trial.suggest_int("width_layer1_vfunc", 4, 128)
    w2_vfunc  = trial.suggest_int("width_layer2_vfunc", 4, 128)
    std_start = trial.suggest_uniform("std_start", 0.7, 1.5)
    std_min   = trial.suggest_uniform("std_min", 0.15, 0.6)
    gamma     = trial.suggest_loguniform("gamma", 0.90, 1e0)
    lmbda     = trial.suggest_loguniform("lambda", 0.01, 1e0)
    polyak_alpha = trial.suggest_loguniform("polyak_alpha", 0.97, 1e0)

    pi = GaussianPolicy([idim, w1_pi, odim], std_start=std_start, std_min=std_min, clip_range=[-2, 2])
    vfunc = ValueFunctionMLP([idim, w1_vfunc, w2_vfunc, 1])
    tvfunc = copy.deepcopy(vfunc)
    opt = torch.optim.Adam(pi.parameters(), lr=lr_policy)
    copt = torch.optim.Adam(vfunc.parameters(), lr=lr_vfunc)

    return train(eps, pi, vfunc, tvfunc, opt, copt, gamma, lmbda, polyak_alpha, 'ppo_pendulum_')


In [9]:
def train(eps, pi, vfunc, tvfunc, opt, copt, gamma, lmbda, polyak_alpha, study_name, debug=False):
    rew_set = False
    rew_ma = None
    alpha = 1-1/15
    if debug:
        wll.new()
        writer = wll.writer
    for ep in range(0, eps):
        l = envs.list_rollout(pi.act, gamma=gamma, length=10)
        rss = l[0][2].mean().item()
        adv = list_gae(l, tvfunc, gamma, lmbda=lmbda)
        add_list2list(adv, l)
        rbuff = ReplayBuffer(nitems=10, max_len=100000)
        rbuff.list_add(l)

        for imini_batch in range(18):
            oldobs, a, r, obs, d, oldprobs, smpls, muold, stdold, gae = rbuff.get(len(rbuff)//6)
            adv = gae.detach() - vfunc(oldobs)
            adv = ((adv-adv.mean())/(adv.std()+1e-8)).detach()
            for optstep in range(2):
                _, (newprobs, _, mu, std) = pi.act(oldobs, smpl=smpls)
                p = torch.distributions.Normal(muold, stdold)
                q = torch.distributions.Normal(mu, std)
                kldiv = torch.distributions.kl_divergence(p, q)
                if kldiv.mean().item() > 0.03:
                    print("stopppp", kldiv.mean().item(), optstep)
                    break
                ratio = torch.exp(newprobs-oldprobs)
                e = 0.15
                clipped = (ratio != torch.clamp(ratio, 1-e, 1+e)).float().mean().detach().item()
                loss = -torch.min(ratio*adv, torch.clamp(ratio, 1-e, 1+e)*adv).mean()
                opt.zero_grad()
                loss.backward()
                # global_gradient_clip(pi)
                opt.step()
            
            for optstep in range(2):                    
                td = ((gae.detach() - vfunc(oldobs))**2).mean()
                copt.zero_grad()
                td.backward()
                # global_gradient_clip(vfunc)
                copt.step()


        polyak(vfunc, tvfunc, polyak_alpha)
        if rew_set:
            rew_ma = alpha*rew_ma+(1-alpha)*r.mean().item()
        else:
            rew_ma = r.mean().item()
            rew_set = True
        print(ep, end='\r')

        if debug:
            if ep%1==0:
                print(ep, loss.item(), "r", rss, " td", td.item(), "clip", clipped, "___", pi.last_std.mean().item())
            if ep%20==0:
                print("showit=",showit(env, pi.act, max_steps=650))
                env.close()
            writer.add_scalar("a/clipped", clipped, ep)
            writer.add_scalar("a/clipped", clipped, ep)
            writer.add_scalar("a/loss", loss.item(), ep)
            writer.add_scalar("a/td", td.item(), ep)
            writer.add_scalar("a/reward", r.mean(), ep)
            writer.add_scalar("a/clipped", clipped, ep)
    rfinal = oldobs, a, r, obs, d, oldprobs, smpls, muold, stdold, gae = rbuff.get(len(rbuff)-1)
    with open('optuna/'+study_name+'.pckl', 'wb') as f:
        pickle.dump(study, f)
    if debug:
        return rew_ma, pi, vfunc
    return -rew_ma


In [12]:
study.optimize(objective_function, n_trials=200, n_jobs=1)

0.017025327625858438, 'polyak_alpha': 0.9873055825706365}. Best is trial 66 with value: 5.33431339263916.
[I 2020-10-24 13:59:01,026] Trial 174 finished with value: 6.937841188192962 and parameters: {'seed': 1, 'lr_policy': 0.0001361565999469788, 'lr_vfunc': 23.403027932180144, 'width_layer1_policy': 90, 'width_layer1_vfunc': 122, 'width_layer2_vfunc': 95, 'std_start': 0.9680687111607704, 'std_min': 0.47841446554868183, 'gamma': 0.9686299563665551, 'lambda': 0.030112676497760258, 'polyak_alpha': 0.9820420856300301}. Best is trial 66 with value: 5.33431339263916.
[I 2020-10-24 14:00:18,103] Trial 175 finished with value: 5.927765783728624 and parameters: {'seed': 1, 'lr_policy': 0.00020479098357992817, 'lr_vfunc': 4.813878808064894, 'width_layer1_policy': 71, 'width_layer1_vfunc': 32, 'width_layer2_vfunc': 117, 'std_start': 0.8482650237675269, 'std_min': 0.5718168826406769, 'gamma': 0.9669846527816939, 'lambda': 0.022704265650466973, 'polyak_alpha': 0.9893457546168543}. Best is trial 66

KeyboardInterrupt: 

In [25]:
print(study.best_params)
study.best_params['lr_policy']
some = study.get_trials()[202].params

{'lr_policy': 0.000392148144561186, 'lr_vfunc': 0.9557335221938151, 'width_layer1_policy': 100, 'width_layer1_vfunc': 189, 'width_layer2_vfunc': 85, 'std_start': 0.6401847525381115, 'gamma': 0.821649998393548, 'lambda': 0.02157171173951722, 'polyak_alpha': 0.7743138632219162}


In [11]:
# study = optuna.create_study(study_name='PPO_Pendulum')
import pickle  
import optuna

with open('optuna/PPO_Pendulum.pckl', 'rb') as f:
    study = pickle.load(f)

In [27]:
def best_f(eps=100):
    from_what = some
    seed = 0
    np.random.seed(seed)
    env.seed(seed)
    torch.manual_seed(seed)
    eps = eps
    lr_policy = from_what["lr_policy"]
    lr_vfunc  = from_what["lr_vfunc"]
    w1_pi     = from_what["width_layer1_policy"]
    #w2_pi     = trial.suggest_int("width_layer2_policy", 1, 64)
    w1_vfunc  = from_what["width_layer1_vfunc"]
    w2_vfunc  = from_what["width_layer2_vfunc"]
    std_start = from_what["std_start"]
    std_min   = from_what["std_min"]
    gamma     = from_what["gamma"]
    lmbda     = from_what["lambda"]
    polyak_alpha = from_what["polyak_alpha"]

    pi = GaussianPolicy([idim, w1_pi, odim], std_start=std_start, std_min=std_min)
    vfunc = ValueFunctionMLP([idim, w1_vfunc, w2_vfunc, 1])
    tvfunc = copy.deepcopy(vfunc)
    opt = torch.optim.Adam(pi.parameters(), lr=lr_policy)
    copt = torch.optim.Adam(vfunc.parameters(), lr=lr_vfunc)

    return train(eps, pi, vfunc, tvfunc, opt, copt, gamma, lmbda, polyak_alpha, "test", debug=True)

pi, vfunc, rfinal = best_f(50)

0 -0.002476269379258156 r -8.483715057373047  td 8261.396484375 clip 0.0 ___ 0.9991866946220398
showit= -1385.4246033660284
1 -0.0004192566266283393 r -6.783168792724609  td 8715.5849609375 clip 0.04694835841655731 ___ 1.0018621683120728
2 0.00026956270448863506 r -8.30062198638916  td 8697.1416015625 clip 0.0 ___ 1.0015404224395752
3 -0.0012082281755283475 r -7.729132652282715  td 8605.3505859375 clip 0.014084506779909134 ___ 0.9997485280036926
4 0.0005795604665763676 r -7.348391532897949  td 8468.6865234375 clip 0.0 ___ 1.0031074285507202
5 0.003671300830319524 r -8.635173797607422  td 8385.259765625 clip 0.009389671497046947 ___ 0.9993189573287964
6 -0.001779148238711059 r -6.813990592956543  td 8266.595703125 clip 0.0 ___ 1.0003961324691772
7 0.0021956951823085546 r -8.187200546264648  td 8157.06103515625 clip 0.03286384791135788 ___ 1.0077035427093506
8 -0.0024069850333034992 r -8.256591796875  td 8090.34375 clip 0.0 ___ 1.0060415267944336
9 -0.00033714459277689457 r -6.8341436386